In [ ]:
from classifier import BertSentClassifier
from types import SimpleNamespace
from tokenizer import BertTokenizer
import torch

In [ ]:
config = {
    'hidden_dropout_prob': 0.3, 
     'num_labels': 2,
     'hidden_size': 768,
     'option': 'flexible'
}

config = SimpleNamespace

model = BertSentClassifier(config)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
model_state_dict = torch.load()